In [7]:
from ortools.sat.python import cp_model
import datetime

# Helper function to check if a room's time slots overlap
def overlap(slot1, slot2):
    start1, end1 = slot1
    start2, end2 = slot2
    return not (end1 <= start2 or end2 <= start1)

# Convert time string to datetime object
def parse_time(time_str):
    return datetime.datetime.strptime(time_str, "%Y-%m-%d %H:%M")

def schedule_rooms(rooms, requests, floor_distance_weight=1, unused_capacity_weight=1):
    model = cp_model.CpModel()

    # Convert rooms and requests into appropriate structures
    room_names = [room['name'] for room in rooms]
    room_capacities = {room['name']: room['type_capacity'] for room in rooms}
    room_floors = {room['name']: room['floor'] for room in rooms}

    # Time slots for each request
    request_slots = {}
    for i, request in enumerate(requests):
        request_slots[i] = request['time_slots']

    # Variables: room assignments
    room_assignments = {}
    for req_idx, request in enumerate(requests):
        room_assignments[req_idx] = []
        for i in range(request['num_rooms']):
            room_assignments[req_idx].append(model.NewIntVar(0, len(rooms) - 1, f"room_{req_idx}_{i}"))

    # Variables for unused capacity
    unused_capacity = {}
    for req_idx, request in enumerate(requests):
        unused_capacity[req_idx] = []
        for i in range(request['num_rooms']):
            room_name = room_names[room_assignments[req_idx][i].Index()]
            room_capacity = room_capacities[room_name].get(request['type'], 0)
            unused_capacity[req_idx].append(model.NewIntVar(0, room_capacity, f"unused_capacity_{req_idx}_{i}"))
    
    # Add constraints:
    # 1. Each request gets exactly the required number of rooms
    for req_idx, request in enumerate(requests):
        for i in range(request['num_rooms']):
            model.Add(room_assignments[req_idx][i] != -1)  # Enforce that the room assignment is valid

    # 2. Room capacity and type matching
    for req_idx, request in enumerate(requests):
        for i in range(request['num_rooms']):
            room_name = room_names[room_assignments[req_idx][i].Index()]
            room_capacity = room_capacities[room_name].get(request['type'], 0)
            model.Add(unused_capacity[req_idx][i] == room_capacity - request['num_people'])

    # 3. Room time slot conflicts
    for req_idx1, request1 in enumerate(requests):
        for req_idx2, request2 in enumerate(requests):
            if req_idx1 != req_idx2:
                for slot1 in request1['time_slots']:
                    for slot2 in request2['time_slots']:
                        for room1 in range(request1['num_rooms']):
                            for room2 in range(request2['num_rooms']):
                                # If rooms overlap and time slots overlap, we can't assign the same room
                                if overlap(slot1, slot2):
                                    model.Add(room_assignments[req_idx1][room1] != room_assignments[req_idx2][room2])

    # 4. Minimize floor distance and unused capacity
    floor_distance = []
    for req_idx, request in enumerate(requests):
        for i in range(request['num_rooms']):
            room_name = room_names[room_assignments[req_idx][i].Index()]
            room_floor = room_floors[room_name]
            # Convert the time string to datetime for calculating the difference
            request_start_time = parse_time(request['time_slots'][0][0])  # Start time of the request
            current_time = datetime.datetime.now()  # You can also use a fixed time for simulation
            
            # Calculate the floor distance
            time_diff_minutes = abs((request_start_time - current_time).total_seconds()) // 60
            floor_distance.append(time_diff_minutes * abs(room_floor))  # Multiply by the floor difference

    # Objective function: Minimize total floor distance and unused capacity
    objective = floor_distance_weight * sum(floor_distance) + unused_capacity_weight * sum([sum(x) for x in unused_capacity.values()])

    model.Minimize(objective)

    # Solve the problem
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        # Output results
        schedule = {}
        for req_idx, request in enumerate(requests):
            schedule[req_idx] = []
            for i in range(request['num_rooms']):
                assigned_room_idx = solver.Value(room_assignments[req_idx][i])
                schedule[req_idx].append(room_names[assigned_room_idx])

        return schedule
    else:
        return "No solution found"

In [ ]:
rooms = [
    {"name": "LES VOÛTES", "type_capacity": {"Theatre": 80, "Reception": 230}, "floor": 0},
    {"name": "ARCHAMBAULT", "type_capacity": {"Theatre": 60, "Reception": 50}, "floor": 0},
    {"name": "LA GALERIE D’ART", "type_capacity": {"Theatre": 100, "Reception": 50}, "floor": 1},
    {"name": "CHEZ PLUME", "type_capacity": {"Theatre": 0, "Reception": 100}, "floor": 1},
    {"name": "S. BERNHARDT", "type_capacity": {"Theatre": 220, "Reception": 320}, "floor": 2},
]

requests = [
    {"type": "Theatre", "num_rooms": 1, "num_people": 80, "time_slots": [["2023-10-01 13:00", "2023-10-01 15:00"]] },
    {"type": "Theatre", "num_rooms": 1, "num_people": 80, "time_slots": [["2023-10-01 14:00", "2023-10-01 15:00"]] },
    {"type": "Reception", "num_rooms": 1, "num_people": 200, "time_slots": [["2023-10-01 15:05", "2023-10-01 19:00"]] },
]

# Call the function with appropriate weights
schedule = schedule_rooms(rooms, requests, floor_distance_weight=1, unused_capacity_weight=0.5)

# Print the schedule result
print(schedule)


No solution found
